#

# The Basics

The chapter will introduce some bare basics.  In order to explain some things, we will need to briefly go over how
lifetimes work.  It's assumed you already followed Chapter 0, the prerequisites.

In [ ]:
let x = 10;
let y = x;
println!("y is {}", y);